In [1]:
%matplotlib inline
from collections import defaultdict
from constants import *
from os import listdir, makedirs
from os.path import join, isfile, exists, dirname
import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from tqdm._tqdm_notebook import tqdm_notebook
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import Word2Vec, Doc2Vec
from utils import load

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

tqdm_notebook.pandas()
pd.options.display.max_columns = 30

In [2]:
# map lemmatization back to unnormalized text
phrases = load('phrases', 'minimal')
phrases
#phrases.groupby('token').value_counts()

Reading ../data/preprocessed/dewiki_phrases_lemmatized.pickle


token
Alan_Smithee                                                                                  Alan_Smithee
Actinium                                                                                          Actinium
ANG_Lee                                                                                            Ang_Lee
Anschluss                                                                                        Anschluss
Aussagenlogik                                                                                Aussagenlogik
Anthony_Minghella                                                                        Anthony_Minghella
us-amerikanisch_Film                                                                US-amerikanischer_Film
Vorsatz_für_Maßeinheit                                                           Vorsätze_für_Maßeinheiten
Abkürzungen/Gesetz_und_Recht                                                 Abkürzungen/Gesetze_und_Recht
Liste_von_Unternehmen_mit_Namen

In [19]:
phrases['gelb_Karte']

'Gelbe_Karte'

In [20]:
def unlemmatize(token):
    try:
        words = phrases[token]
        if words is None:
            #print('not in index', token)
            return token.replace('_', ' ')
        elif isinstance(words, str):
            return words.replace('_', ' ')
        else:
            if token in words.values:
                word = token.replace('_', ' ')
                return word
            else:
                wc = words.value_counts()
                word = wc.index[0].replace('_', ' ')
                #print(token)
                #print(words.value_counts())
                #print('return word with highest count', word)
                return word
    except:
        word = token.replace('_', ' ')
        print('not in index:', word)
        return word

In [21]:
collected = []
for dataset in ['O', 'P', 'E', 'FA', 'FO', 'dewa1']:
    for metric in ['ref', 'c_npmi']:
        params = [dataset, metric, 100, 'e42', 'minimal']
        l = load('labels', *params).applymap(unlemmatize)
        t = load('topics', *params)
        s = load('scores', *params)
        t['cv_score'] = s.c_v_eval
        # wiktionary would be better than wikipedia
        t['count_wiki_titles'] = t.loc[:, 'term0':'term9'].applymap(lambda x: x in phrases.index).sum(axis=1)
        t['cv_rank'] = t.cv_score.rank()
        t['wiki_rank'] = t.count_wiki_titles.rank()
        t['rank'] = t.cv_rank + t.wiki_rank
        t.loc[:, 'term0':'term9'] = t.loc[:, 'term0':'term9'].applymap(unlemmatize)
        t = t.sort_values('rank', ascending=False)
        l = l.iloc[t.index, :]
        collected.append((dataset, metric, t, l))

Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_label-candidates_full.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/OnlineParticipation_evaluation-scores.csv
not in index: Fahrradwrack
not in index: Schlagloch
not in index: Haushalt
not in index: Godesberg
not in index: Sozialwohnung
not in index: Bußgeld
not in index: Müll
not in index: EUR
not in index: Stadtteil
not in index: PKW
not in index: Sparvorschlag
not in index: Grünanlage
not in index: Einsparung
not in index: Beschilderung
not in index: Auto
not in index: Zumutung
not in index: Fahrradwrack
not in index: Schlagloch
not in index: Haushalt
not in index: Godesberg
not in index: Sozialwohnung
not in index: Bußgeld
not in index: Müll
not in index: EUR
not in index: Stadtteil
not in index: PKW
not in index: Sparvorschlag
not in index: Grünanlage
not in index: Einsparung
not in index: Beschilderung
not i

not in index: Und/Oder
not in index: Anderen
not in index: Telekom
not in index: Gürtelausbau
not in index: Bürgersteig
not in index: Bürgervorschlag
not in index: Verstoß
not in index: Verkehrssituation
not in index: Sitzgelegenheit
not in index: Verbindungsweg
not in index: Jugendliche
not in index: Radwegenetz
not in index: Verantwortliche
not in index: Wegfall
not in index: Severinsweg
not in index: Anwohner
not in index: Autofahrer
not in index: Bewohner
not in index: Ehrenfeldgürtel
not in index: Autoverkehr
not in index: Fußgängerampel
not in index: Fehlender
not in index: Fahrradfahrer
not in index: Stadtteilbibliothek
not in index: Halteverbot
not in index: Allgemeinheit
not in index: Nordseite
not in index: Bebauung
not in index: Evtl
not in index: Stadtteil
not in index: Rheinufer
not in index: Ausweitung
not in index: Überweg
not in index: Bereich
not in index: Betrag
not in index: Koblenzer Straße
not in index: Investor
not in index: Anteil
not in index: Schulkind
not in i

not in index: Kollegin
not in index: Nichtverbreitung
not in index: Austausch
not in index: Vereinte Nation
not in index: Kulturstaatsminister
not in index: Teilung
not in index: Provenienzrecherche
not in index: Lehrerin
not in index: Ostdeutscher
not in index: Gedenken
not in index: Vielfalt
not in index: Sammlung
not in index: Atomwaffe
not in index: Anlass
not in index: 3 . Oktober
not in index: Bergen-Belsen
not in index: Würde
not in index: Denken
not in index: Auswärtige Kultur- und Bildungspolitik
not in index: Kulturerbe
not in index: Menschenrechtsrat
not in index: Kinofilm
not in index: Präsidentschaft
not in index: Zusammenarbeit
not in index: Mitgliedsstaat
not in index: Mio.
not in index: Studentin
not in index: Bürgerin
not in index: Ehrenamtliche
not in index: Osze-Vorsitz
not in index: Sed-Diktatur
not in index: Kollegin
not in index: Kollegin
not in index: Verhältnis
not in index: Überlebende
not in index: Haushalt
not in index: Nationalsozialist
not in index: Putin
n

not in index: Bereich
not in index: Inselregion
not in index: Übereinkommen
not in index: Zusammenarbeit
not in index: Präsidentin
not in index: Ziffer
not in index: Kommission
not in index: Kommission
not in index: Putin
not in index: Vielfalt
not in index: Zusammenarbeit
not in index: Kollegin
not in index: Fischereiabkomme
not in index: Bereich
not in index: EZB
not in index: Antrag
not in index: Atomwaffe
not in index: Präsidentschaft
not in index: Bekämpfung
not in index: Änderung
not in index: Bestrafung
not in index: Sitzungsperiode
not in index: Tabakanbau
not in index: Bürgerin
not in index: BSE
not in index: OLAF
not in index: Zusammenarbeit
not in index: Erkrankung
not in index: 7 .
not in index: Fischereisektor
not in index: Milosevic
not in index: GVO
not in index: Nichtverbreitung
not in index: Änderungsantrag
not in index: Bereich
not in index: Beitritt
not in index: Zusammenhalt
not in index: Präsidentin
not in index: Kommission
not in index: Einwanderer
not in index: U

not in index: Schulz
not in index: Verteidigungsministerin
not in index: Autofahrer
not in index: Anteil
not in index: NDR
not in index: Grundgesetz
not in index: Attentäter
not in index: Schauspielerin
not in index: Imame
not in index: Leser
not in index: Fillon
not in index: Studierender
not in index: Premierministerin
not in index: Sammlung
not in index: km/h
not in index: FPÖ
not in index: Chinese
not in index: Lastwagen
not in index: Befragte
not in index: Ermittler
not in index: Özdemir
not in index: Homosexuelle
not in index: Airline
not in index: weiß Haus
not in index: Verbleib
not in index: Assad
not in index: Stundenkilometer
not in index: Bobic
not in index: Separatist
not in index: Autohersteller
not in index: Akku
not in index: Giftanschlag
not in index: Strafzoll
not in index: Interessantes
not in index: Özil
not in index: Vorjahr
not in index: Ermittlung
not in index: Journalistin
not in index: Gesetzgeber
not in index: Niki
not in index: Hersteller
not in index: Anbiet

not in index: Leitindex
not in index: VW
not in index: Telekom
not in index: Forscher
not in index: Leser
not in index: Kanzlerin
not in index: km/h
not in index: Liberaler
not in index: Überschuss
not in index: Ermittlung
not in index: Braut
not in index: Sicherheitsrisiko
not in index: US-Präsident
not in index: US-Regierung
not in index: Grieche
not in index: Airline
not in index: OLG
not in index: Ermittlung
not in index: Sammlung
not in index: Zuschauer
not in index: Tote
not in index: Investor
not in index: Vorjahr
not in index: Bundesinnenminister
not in index: Tochter
not in index: Erkrankung
not in index: EU-Staat
not in index: Schweiger
not in index: Sprecherin
not in index: Angreifer
not in index: Tote
not in index: Bußgeld
not in index: Ermittler
not in index: Beatles
not in index: 4 . Juli
not in index: Hälfte
not in index: Staatschef
not in index: Tropensturm
not in index: Zunahme
not in index: Kombi
not in index: Feier
not in index: Autorin
not in index: Brexit
not in in

not in index: FAZ.NET
not in index: Steuerpflichtige
not in index: KPÖ
not in index: Mitgliedschaft
not in index: Zusammenarbeit
not in index: 6 .
not in index: Umstellung
not in index: 3 .
not in index: Rückkehr
not in index: Grenzwert
not in index: Vatikan
not in index: Komponist
not in index: Anbieter
not in index: CDU/CSU
not in index: Rückgabe
not in index: Job Bielefeld
not in index: SPÖ
not in index: Bearbeit
not in index: Vorstandsmitglied
not in index: Kindertageseinrichtung
not in index: cm
not in index: Schnitzer
not in index: Abs.2
not in index: BGB


In [22]:
for tpl in collected:
    variant = 'default' if tpl[1] == 'ref' else 'rerank'
    file = f'{DATASETS[tpl[0]]}_{variant}'
    tpl[2].to_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', file + '_topics.csv'))
    tpl[3].to_csv(join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', file + '_labels.csv'))

In [4]:
dataset, metric = 'dewac1', 'ref'
params = [dataset, metric, 100, 'e42', 'minimal']
l = load('labels', *params).applymap(unlemmatize)
t = load('topics', *params)
s = load('scores', *params)
t['cv_score'] = s.c_v_eval
# wiktionary would be better than wikipedia
t['count_wiki_titles'] = t.loc[:, 'term0':'term9'].applymap(lambda x: x in phrases.index).sum(axis=1)
t['cv_rank'] = t.cv_score.rank()
t['wiki_rank'] = t.count_wiki_titles.rank()
t['rank'] = t.cv_rank + t.wiki_rank
t.loc[:, 'term0':'term9'] = t.loc[:, 'term0':'term9'].applymap(unlemmatize)
t = t.sort_values('rank', ascending=False)
t

Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_label-candidates_full.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_topic-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_evaluation-scores.csv


,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9,cv_score,count_wiki_titles,cv_rank,wiki_rank,rank
24,Tier,Hund,Jäger,Jagd,Katze,Fuchs,Vogel,Mensch,Wald,Haustier,0.825688,10,93.0,78.5,171.5
12,Musik,Band,Bühne,Publikum,Lied,Konzert,Musiker,Theater,Künstler,Stück,0.809177,10,91.0,78.5,169.5
68,Artikel,Stern,Erde,Planet,KM,Link,Komet,Quell,NASA,Sonne,0.795549,10,88.0,78.5,166.5
90,Gott,Jesus,Herr,Mensch,Glaube,Leben,Christus,Christ,Volk,Vater,0.775762,10,85.0,78.5,163.5
45,Uhr,Quell,Dienstag,Freitag,Schröder,Montag,Spiegel,Mittwoch,Donnerstag,Samstag,0.757976,10,83.0,78.5,161.5
6,Kirche,Beamter,Bischof,Papst,Zeit,Kardinal,katholisch Kirche,Rom,Priester,Erzbischof,0.740925,10,78.0,78.5,156.5
32,Krieg,USA,Volk,Staat,Irak,Land,Welt,Regierung,Friede,Europa,0.726734,10,75.0,78.5,153.5
30,Deutschland,Jude,Deutsch,Hitler,Opfer,Berlin,Nazi,Pol,Krieg,Nationalsozialismus,0.716404,10,71.0,78.5,149.5
58,Akademie,Potsdam,Sachs,Magdeburg,Akademie der Wissenschaft,Kirche,Sachsen-Anhalt,Thüring,Erfurt,Brandenburg,0.703753,10,69.0,78.5,147.5
83,Prozent,Euro,Bank,Unternehmen,Million,Geld,Anleger,Deutschland,Fond,Aktie,0.690818,10,67.0,78.5,145.5


In [5]:
l.iloc[t.index, :]

,label0,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
24,Wildschwein,Wildtier,Hauskatze,Tayra,Schaf,Nutztier,klein indisch Zibetkatze,Kulturflüchter,Kleinsäuger,Haustier,Dingo,Kusus,Riesenbeutelmarder,Haushund,Beutegreifer,Lungenwurm,Kaninchen,Fleischfresser,indisch Gazelle,Pampashirsch
12,Duett,Begleitband,Aufführung,Konzert,Tournee,klassisch Musik,Rockband,Auftritt,Bühnenwerk,Show,Singer-Songwriter,Bühnenmusik,Darsteller,Musik sein,Arrangeur,Liedermacher,Gastspiel,Soundtrack,Gitarrist,Musiker
68,Gestirn,Sonnensystem,Mond,Planetensystem,interplanetar Staub,Raumsonde,Umlaufbahn,Infrarotgalaxie,Standardkerze,Umlaufzeit,interstellar Medium,Exoplanet,Ross 128 B,lokal Blas,Sternstrom,Hipparcos-Katalog,Himmelskörper,Epsilon Eridani,Sternhaufen,Trappist-1
90,Jesus Christus,Gott,Wort Gott,Sohn Gott,Leib Christus,Anbetung,Bekehrung,Gottesfurcht,Christentum,heilig Schrift,Evangelium,Glaubensbekenntnis,Reich Gott,Glaube,himmlisch König,Sünde,Abrahams Schoß,Offenbarungsreligion,Heiland,Heilige
45,Sonntagabend,Freitag,Donnerstag,Dienstag,Sonntag,Samstag,Mittwoch,Wochenende,Feiertag,Vormittag,Tulpensonntag,heute mittag,Hallo Niedersachse,Wochentag,Werktag,Servicezeit,RTL Nachtjournal,Sat.1 Nachricht,Zdf-Mittagsmagazin,heute – in Europa
6,päpstlich Legat,Weihbischof,Kurienbischof,Erzbistum Turin,Suffraganbischof,Erzbischof,Erzpriester,Diözesanbischof,Bischof,apostolisch Delegat,Metropolit,Dompropst,Pontifikat,Papst,Bischofssitz,Offizial,Diözese,Generalvikar,Kathedrale,James Robert Knox
32,vereinigen Staat,Staat,Siegermacht,Militärbündnis,westlich Welt,Regierungschef,Land,Nachfolgestaat,Kolonialmacht,Landmacht,Supermacht,Nationalstaat,Iran,allgemein Gewaltverbot,Westeuropa,nah Osten,Mutterland,Monarchie,arabisch Welt,Syrien
30,Zeit der Nationalsozialismus,Holocaust,Antisemitismus,dritte Reich,Überfall auf Pol,Volksdeutscher,Nationalsozialismus,Anschluss Österreich,zweite Weltkrieg,Novemberpogrom 1938,Adolf Hitler,protestantisch Rompilger,volksdeutscher Selbstschutz,polnisch Aufstand,Nazi,Konzentrationslager,Machtergreifung,nürnberger Gesetz,nationalsozialistisch Propaganda,Besatzungsmacht
58,Thüring,Merseburg,Erfurt,Sachsen-Anhalt,Provinz Sachs,Brandenburg,Jena,Leipzig,Magdeburg,Naumburg,Land Brandenburg,Brandenburg an der Havel,Quedlinburg,Halberstadt,Cottbus,Dietzhausen,Gera,Nordhausen,Chemnitz,Bernburg
83,One Equity Partner,Geldmarktfond,Immobilienfond,Cinven,Startkapital,BC Partner,EQT,Zweitmarkt,Investmentgesellschaft,Advent International,Aktionär,Gazprombank,Refco,Singapore Exchang,Grundkapital,Bankensektor,Beteiligungsgesellschaft,Investmentfond,Die Bank,Pensionsfond


In [7]:
l = load('labels', 'dewac1', 100, 'e42', 'c_npmi', 'minimal')
l

('labels', 'dewac1', 100, 'e42', 'c_npmi', 'small')
labels
dewac_nbfiles01
Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_label-candidates_full.csv


,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
0,Kommunismus,demokratisch_Sozialismus,Sozialdemokratie,Diktatur_der_Proletariat,sozialistisch_Partei,Sozialismus,Marxismus,bürgerlich_Partei,Arbeiterpartei,gothaer_Programm,Staatssozialismus,Sozialfaschismusthese,kommunistisch_Partei_Deutschland,Leninismus,sozialistisch_Demokratie,Einheitsfront,antiautoritär_Sozialismus,Arbeiterbewegung,Marxismus-Leninismus
1,Stiefmutter,Hedwig,Pittheu,Stiefvater,Schwester,Adelheid,Anna,Johann_von_England,Igrain,Eriphyle,Andrea_von_Ungar,Argeia,Der_Vater,Tochter_der_König,Fürstin,Anna_von_Böhme,Enkel,Großvater,Cousine
2,Lebenspartnerschaftsgesetz,gleichgeschlechtlich,Verpartnerung,Lebensformenpolitik,lesbisch,Homosexualität,Scheidung,Eherecht,Lebensgemeinschaft,Regenbogenfamilie,Eingetragene_Partnerschaft,Ehescheidung,Gütertrennung,Abstandsgebot,Zusammenleben,sexuell_Identität,Partnerschaftsgesetz,Adoption,kirchlich_Trauung
3,Personalvertretungsgesetz,Tarifvertrag,Bundespersonalvertretungsgesetz,Beschäftigte,Betriebsrat,Gesamtbetriebsrat,Tarifvertragspartei,Sprecherausschuss,Arbeitnehmervertretung,Drittelbeteiligungsgesetz,Belegschaft,sozial_Angelegenheit,Arbeitsrecht,Betriebsverfassung,Betriebsrentengesetz,Arbeitsverhältnis,Betriebsbuße,Betriebsänderung,Arbeitsvertrag
4,Aktiengesellschaft,Geschäftsanteil,Grundkapital,Kapitalerhöhung,Stammkapital,Gesellschafter,Gesellschafterversammlung,Anteilseigner,Vorstandsvorsitzende,Stammaktie,Hauptversammlung,Kommanditgesellschaft_auf_Aktie,Generalversammlung,Aufsichtsrat,Gesellschaft_mit_beschränkt_Haftung,Gewerkenversammlung,Kommanditist,Vorzugsaktie,Beteiligungsgesellschaft
5,Verfassung,Staatsrat,Legislative,Nationalversammlung,Volksvertretung,Verfassungsänderung,Gesetzeskraft,Gesetzgebung,Senat,Gesetzentwurf,Staatsgerichtshof,Generalstaatsanwalt,gemeinsam_Ausschuss,Gesetzgebungsverfahren,Abgeordnete,Landesverfassung,Verfassung_der_Land_Baden-Württemberg,Parlament,Verfassungsreform
6,Weihbischof,Kurienbischof,Erzbistum_Turin,Suffraganbischof,Diözesanbischof,Bischof,Erzpriester,Papst,Erzbischof,apostolisch_Delegat,Generalvikar,Kardinal,Dompropst,Metropolit,Diözese,heilig_Stuhl,Koadjutor,Titelkirche,Offizial
7,sofortig_Beschwerde,Prozessgericht,Endurteil,Angeklagte,Zulässigkeit,Anhörung,Hauptverfahren,Teilurteil,Gerichtsverfahren,Prozessurteil,Haftbeschwerde,Hauptsacheverfahren,Sprungrevision,Einspruch,Rechtsmittel,sofortig_Vollziehung,Ablehnungsbescheid,Zwischenurteil,Urteil
8,Strafsache,Verdächtige,Strafkammer,Hauptverfahren,Generalstaatsanwaltschaft,Kläger,Staatsanwaltschaft,Gerichtsverfahren,Tatverdacht,Strafverfolgung,Generalstaatsanwalt,Ermittlungsverfahren,Staatsschutzkammer,Untersuchungsrichter,Freispruch,Staatsanwalt,Beschuldigter,vorsätzlich_Tötung,Strafrichter
9,Wehrpflicht,weiß_Jahrgang,Soldat_auf_Zeit,Wehrersatzdienst,Einberufung,freiwillig_Wehrdienst,Ersatzreserve,Berufssoldat,Zivildienstgesetz,Soldatengesetz,Totalverweigerung,Präsenzdienst,Wehrpflicht_in_Deutschland,Reserveoffizier,Reserveoffizieranwärter,Kriegsdienstverweigerung,Wehrsold,Wehrsoldgesetz,Unteroffizier_auf_Zeit


In [28]:
lx = l.applymap(unlemmatize)
lx

immer
Immer    2
Name: text, dtype: int64
return word with highest count Immer
immer
Immer    2
Name: text, dtype: int64
return word with highest count Immer
Departamento_SAN_Martín
Departamento_San_Martín    5
Name: text, dtype: int64
return word with highest count Departamento San Martín
Kind
Kinder    4
Name: text, dtype: int64
return word with highest count Kinder
Thüring
Thüringen    2
Name: text, dtype: int64
return word with highest count Thüringen
alt_Haus
Altes_Haus       1
Ältestes_Haus    1
Name: text, dtype: int64
return word with highest count Altes Haus
Heinrich_i.
Heinrich_I.    62
Name: text, dtype: int64
return word with highest count Heinrich I.
europäisch_Union
Europäische_Union    2
Name: text, dtype: int64
return word with highest count Europäische Union
wir
Wir    4
Name: text, dtype: int64
return word with highest count Wir
Handball-Weltmeisterschaft_der_Mann_2011/Kader
Handball-Weltmeisterschaft_der_Männer_2011/Kader    24
Name: text, dtype: int64
return word wi

,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
0,Kommunismus,Demokratischer Sozialismus,Sozialdemokratie,Diktatur des Proletariats,Sozialistische Partei,Sozialismus,Marxismus,Bürgerliche Partei,Arbeiterpartei,Gothaer Programm,Staatssozialismus,Sozialfaschismusthese,Kommunistische Partei Deutschlands,Leninismus,Sozialistische Demokratie,Einheitsfront,Antiautoritärer Sozialismus,Arbeiterbewegung,Marxismus-Leninismus
1,Stiefmutter,Hedwig,Pittheus,Stiefvater,Schwester,Adelheid,Anna,Johann von England,Igraine,Eriphyle,Andreas von Ungarn,Argeia,Der Vater,Töchter des Königs,Fürstinnen,Anna von Böhmen,Enkeln,Großvater,Cousine
2,Lebenspartnerschaftsgesetz,Gleichgeschlechtlich,Verpartnerung,Lebensformenpolitik,Lesbisch,Homosexualität,Scheidung,Eherecht,Lebensgemeinschaft,Regenbogenfamilie,Eingetragene Partnerschaft,Ehescheidung,Gütertrennung,Abstandsgebot,Zusammenleben,Sexuelle Identität,Partnerschaftsgesetz,Adoption,Kirchliche Trauung
3,Personalvertretungsgesetz,Tarifvertrag,Bundespersonalvertretungsgesetz,Beschäftigter,Betriebsrat,Gesamtbetriebsrat,Tarifvertragspartei,Sprecherausschuss,Arbeitnehmervertretung,Drittelbeteiligungsgesetz,Belegschaft,Soziale Angelegenheiten,Arbeitsrecht,Betriebsverfassung,Betriebsrentengesetz,Arbeitsverhältnis,Betriebsbuße,Betriebsänderung,Arbeitsvertrag
4,Aktiengesellschaft,Geschäftsanteil,Grundkapital,Kapitalerhöhung,Stammkapital,Gesellschafter,Gesellschafterversammlung,Anteilseigner,Vorstandsvorsitzender,Stammaktie,Hauptversammlung,Kommanditgesellschaft auf Aktien,Generalversammlung,Aufsichtsrat,Gesellschaft mit beschränkter Haftung,Gewerkenversammlung,Kommanditist,Vorzugsaktie,Beteiligungsgesellschaft
5,Verfassung,Staatsrat,Legislative,Nationalversammlung,Volksvertretung,Verfassungsänderung,Gesetzeskraft,Gesetzgebung,Senat,Gesetzentwurf,Staatsgerichtshof,Generalstaatsanwalt,Gemeinsamer Ausschuss,Gesetzgebungsverfahren,Abgeordneter,Landesverfassung,Verfassung des Landes Baden-Württemberg,Parlament,Verfassungsreform
6,Weihbischof,Kurienbischof,Erzbistum Turin,Suffraganbischof,Diözesanbischof,Bischof,Erzpriester,Papst,Erzbischof,Apostolischer Delegat,Generalvikar,Kardinal,Dompropst,Metropolit,Diözese,Heiliger Stuhl,Koadjutor,Titelkirche,Offizial
7,Sofortige Beschwerde,Prozessgericht,Endurteil,Angeklagter,Zulässigkeit,Anhörung,Hauptverfahren,Teilurteil,Gerichtsverfahren,Prozessurteil,Haftbeschwerde,Hauptsacheverfahren,Sprungrevision,Einspruch,Rechtsmittel,Sofortige Vollziehung,Ablehnungsbescheid,Zwischenurteil,Urteil
8,Strafsache,Verdächtiger,Strafkammer,Hauptverfahren,Generalstaatsanwaltschaft,Kläger,Staatsanwaltschaft,Gerichtsverfahren,Tatverdacht,Strafverfolgung,Generalstaatsanwalt,Ermittlungsverfahren,Staatsschutzkammer,Untersuchungsrichter,Freispruch,Staatsanwalt,Beschuldigter,Vorsätzliche Tötung,Strafrichter
9,Wehrpflicht,Weißer Jahrgang,Soldat auf Zeit,Wehrersatzdienst,Einberufung,Freiwilliger Wehrdienst,Ersatzreserve,Berufssoldat,Zivildienstgesetz,Soldatengesetz,Totalverweigerung,Präsenzdienst,Wehrpflicht in Deutschland,Reserveoffizier,Reserveoffizieranwärter,Kriegsdienstverweigerung,Wehrsold,Wehrsoldgesetz,Unteroffizier auf Zeit


In [31]:
tx = t.applymap(unlemmatize)
tx

not in index: Lebenspartnerschaft
not in index: BetrVG
Thoma
Thomas    8
Name: text, dtype: int64
return word with highest count Thomas
not in index: SPD
not in index: Dr.
Kind
Kinder    4
Name: text, dtype: int64
return word with highest count Kinder
not in index: Abs.
not in index: Jugendliche
not in index: Abs.
not in index: Nr.
not in index: Abs.
not in index: Lebenspartnerschaft
not in index: BetrVG
Thoma
Thomas    8
Name: text, dtype: int64
return word with highest count Thomas
not in index: SPD
not in index: Dr.
Kind
Kinder    4
Name: text, dtype: int64
return word with highest count Kinder
not in index: Abs.
not in index: Jugendliche
not in index: Abs.
not in index: Nr.
not in index: Abs.
not in index: Antrag
not in index: Studierender
not in index: Prof.
not in index: Nachhilfeunterricht
Quell
Quelle    6
Name: text, dtype: int64
return word with highest count Quelle
not in index: EStG
Kind
Kinder    4
Name: text, dtype: int64
return word with highest count Kinder
not in index

term0  \
dataset metric param_id nb_topics topic_idx                        
dewac   c_npmi e42      100       0                       Partei   
                                  1                        König   
                                  2          Lebenspartnerschaft   
                                  3                       BetrVG   
                                  4                 Aufsichtsrat   
                                  5                      Artikel   
                                  6                       Kirche   
                                  7                 Entscheidung   
                                  8                      Richter   
                                  9                  Zivildienst   
                                  10                     München   
                                  11               Studiengebühr   
                                  12                       Musik   
                                  13                       Blüte   
                                  14                 Unternehmen   
                                  15                      Thomas   
                                  16                         SPD   
                                  17                        Erbe   
                                  18                       Leute   
                                  19                  Richtlinie   
                                  20                       Frage   
                                  21                       Spiel   
                                  22                       Leben   
                                  23                         Dr.   
                                  24                        Tier   
                                  25                      Mensch   
                                  26                        Land   
                                  27                        Wahl   
                                  28                        Ring   
                                  29                 Hausaufgabe   
...                                                          ...   
                                  70                 Jugendliche   
                                  71                        Abs.   
                                  72                     Patient   
                                  73                     Vertrag   
                                  74                       Fisch   
                                  75                      Witiko   
                                  76                        Film   
                                  77                      Aktion   
                                  78                         Nr.   
                                  79                      Wasser   
                                  80                        Sekt   
                                  81                   Ausländer   
                                  82                     Seminar   
                                  83                        Euro   
                                  84                      Kläger   
                                  85                   Ernährung   
                                  86                 Deutschland   
                                  87                     Polizei   
                                  88                        Wort   
                                  89                        Satz   
                                  90                        Gott   
                                  91                       Pferd   
                                  92                      Türkei   
                                  93                     Projekt   
                                  94                      Schule   
                                  95              Verwaltungsrat   
                                  96           

In [17]:
p['Faraman']

Glaube     Glaube
Glaube    Glauben
Name: text, dtype: object

In [2]:
phrases = load('phrases')
dewiki_links = load('links')
dewiki_links = load('categories')

Reading ../data/preprocessed/dewiki_phrases_lemmatized.pickle
Reading ../data/preprocessed/dewiki_links.pickle
Reading ../data/preprocessed/dewiki_categories.pickle


In [10]:
full = load('labels', 'dewac1')
full

Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_label-candidates_full.csv


label0  \
  dataset metric param_id nb_topics topic_idx Unnamed: 6                                                      
0 dewac   ref    a42      10        0         d2v         ('demokratisch_Partei_der_Sandžak', 0.39613741...   
                                              w2v                            ('Staat', 0.39068081974983215)   
                                              comb                   ('Regierungschef', 0.6574942767620087)   
1 dewac   ref    a42      10        1         d2v         ('bosnisch-herzegowinisch_Fußball-Cup_2010/11'...   
                                              w2v                            ('Stunde', 0.3673739433288574)   
                                              comb                          ('Abpfiff', 0.6062263250350952)   
2 dewac   ref    a42      10        2         d2v                         ('Kalchinia', 0.3716656565666199)   
                                              w2v                            ('Mutter', 0.4431721270084381)   
                                              comb                          ('Pittheu', 0.6014149188995361)   

                                                                                                     label1  \
  dataset metric param_id nb_topics topic_idx Unnamed: 6                                                      
0 dewac   ref    a42      10        0         d2v         ('Partija_demokratskog_progresa', 0.3873167335...   
                                              w2v                             ('Land', 0.37696459889411926)   
                                              comb                 ('vereinigen_Staat', 0.6186474859714508)   
1 dewac   ref    a42      10        1         d2v               ('Dfb-Ligapokal_2004', 0.35411325097084045)   
                                              w2v                             ('Woche', 0.3516615629196167)   
                                              comb                         ('Halbzeit', 0.6003850102424622)   
2 dewac   ref    a42      10        2         d2v                ('Lelex_aus_Troizen', 0.37073785066604614)   
                                              w2v                             ('Frau', 0.41525083780288696)   
                                              comb                          ('Jüngling', 0.588527649641037)   

                                                                                                     label2  \
  dataset metric param_id nb_topics topic_idx Unnamed: 6                                                      
0 dewac   ref    a42      10        0         d2v         ('Liste_der_Außenminister_Portugal', 0.3862513...   
                                              w2v                  ('vereinigen_Staat', 0.3664394021034241)   
                                              comb                    ('Mitgliedstaat', 0.5996586680412292)   
1 dewac   ref    a42      10        1         d2v                      ('WHL_1971/72', 0.35091739892959595)   
                                              w2v                           ('Minute', 0.35051560401916504)   
                                              comb                    ('buzzer_Beater', 0.5556163340806961)   
2 dewac   ref    a42      10        2         d2v                    ('Rebecca_Brown', 0.36863115429878235)   
                                              w2v                        ('Schwester', 0.39976707100868225)   
                                              comb                       ('Stiefvater', 0.5765034556388855)   

                                                                                               label3  \
  dataset metric param_id nb_topics topic_idx Unnamed: 6                                                
0 dewac   ref    a42      10        0         d2v               ('Helmut_Elsler', 0.3854132294654846)   
                                              w2v                   ('Regierung', 0.3627217411994934)   
               

In [11]:
full.index = full.index.droplevel(0).rename(names='label_method', level=-1)
full

label0  \
dataset metric param_id nb_topics topic_idx label_method                                                      
dewac   ref    a42      10        0         d2v           ('demokratisch_Partei_der_Sandžak', 0.39613741...   
                                            w2v                              ('Staat', 0.39068081974983215)   
                                            comb                     ('Regierungschef', 0.6574942767620087)   
                                  1         d2v           ('bosnisch-herzegowinisch_Fußball-Cup_2010/11'...   
                                            w2v                              ('Stunde', 0.3673739433288574)   
                                            comb                            ('Abpfiff', 0.6062263250350952)   
                                  2         d2v                           ('Kalchinia', 0.3716656565666199)   
                                            w2v                              ('Mutter', 0.4431721270084381)   
                                            comb                            ('Pittheu', 0.6014149188995361)   

                                                                                                     label1  \
dataset metric param_id nb_topics topic_idx label_method                                                      
dewac   ref    a42      10        0         d2v           ('Partija_demokratskog_progresa', 0.3873167335...   
                                            w2v                               ('Land', 0.37696459889411926)   
                                            comb                   ('vereinigen_Staat', 0.6186474859714508)   
                                  1         d2v                 ('Dfb-Ligapokal_2004', 0.35411325097084045)   
                                            w2v                               ('Woche', 0.3516615629196167)   
                                            comb                           ('Halbzeit', 0.6003850102424622)   
                                  2         d2v                  ('Lelex_aus_Troizen', 0.37073785066604614)   
                                            w2v                               ('Frau', 0.41525083780288696)   
                                            comb                            ('Jüngling', 0.588527649641037)   

                                                                                                     label2  \
dataset metric param_id nb_topics topic_idx label_method                                                      
dewac   ref    a42      10        0         d2v           ('Liste_der_Außenminister_Portugal', 0.3862513...   
                                            w2v                    ('vereinigen_Staat', 0.3664394021034241)   
                                            comb                      ('Mitgliedstaat', 0.5996586680412292)   
                                  1         d2v                        ('WHL_1971/72', 0.35091739892959595)   
                                            w2v                             ('Minute', 0.35051560401916504)   
                                            comb                      ('buzzer_Beater', 0.5556163340806961)   
                                  2         d2v                      ('Rebecca_Brown', 0.36863115429878235)   
                                            w2v                          ('Schwester', 0.39976707100868225)   
                                            comb                         ('Stiefvater', 0.5765034556388855)   

                                                                                               label3  \
dataset metric param_id nb_topics topic_idx label_method                                                
dewac   ref    a42      10        0         d2v                 ('Helmut_Elsler', 0.3854132294654846)   
                                            w2v                     ('Regierung', 0.3627217411994934)   
               

In [17]:
'dataset' in full.index.names

True

In [14]:
simple = (
    full[full.index.get_level_values(-1) == 'comb']
    .reset_index(drop=True)
    .applymap(lambda x: eval(x)[0])
)
simple

,label0,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
0,Regierungschef,vereinigen_Staat,Mitgliedstaat,Innenpolitik,Land,Volkspartei_Finnland,Staat,Partido_POR_la_Democracia,Abkomme,Wahl_1993,Leichtathletik-Europameisterschaft_2010/Teilne...,serbisch_Regierung,politisch_Bewegung,Regierung,Bündnis,liberal_Partei,Außenminister,Nationalstaat,Bundesregierung,Exilregierung
1,Abpfiff,Halbzeit,buzzer_Beater,Feldspieler,Stunde,Spielzug,Regelschwellung,Minute,Anpfiff,Pflichtspiel,Tauchzeit,Präpatenz,Tag,Startaufstellung,Nachmittag,Eigentor,Nachspielzeit,Hattrick,annehmbar_Anwenderexposition,Umdrehung
2,Pittheu,Jüngling,Stiefvater,Kind,Mädchen,Magd,Leid,Oichaliud,Mutter,Bruder,Trost,Sarah,Person,Freund,Frau,Witwer,Jeannie,Der_Vater,Mann,Schwester


In [77]:
file = join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'dewac_nbfiles01_topic-candidates.csv')
topics = pd.read_csv(file)
topics = (
    topics
    .query('metric == "ref" and param_id == "e42" and nb_topics == 100')
    .drop(topics.columns[:5], axis=1)
    .reset_index(drop=True)
    .iloc[:20, :]
)
topics

,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9
0,Partei,PDS,Arbeiter,Kampf,Kommunist,DDR,Sozialismus,Gewerkschaft,Genosse,Kraft
1,König,Frau,Mann,Mädchen,Schiff,Hand,Wasser,Märchen,Tag,Vater
2,Lebenspartnerschaft,Ehe,Lebenspartner,Schwuler,Partner,Lesbe,Ehegatte,Abs.,Paar,Partnerschaft
3,BetrVG,Betriebsrat,Abs.,Nr.,Betrieb,OLG,Arbeitgeber,ABR,Mitbestimmungsrecht,Arbeitgeberin
4,Gesellschaft,Aufsichtsrat,Unternehmen,Vorstand,Aktionär,Aktie,Hauptversammlung,AktG,Kodex,Vertrauen
5,Artikel,Gesetz,Art,Mitglied,König,Verfassung,Recht,Präsident,Staat,Regierung
6,Kirche,Beamter,Bischof,Papst,Zeit,Kardinal,katholisch_Kirche,Rom,Priester,Erzbischof
7,Entscheidung,Antrag,Verfahren,Gericht,Fall,Grund,Satz,Antragsteller,Behörde,Anordnung
8,Richter,Staatsanwaltschaft,Fall,Aussage,Gericht,Beweis,Dr.,Impfung,Herr,Prozess
9,Zivildienst,Wehrpflicht,Bundeswehr,Wehrpflichtiger,Monat,Dienst,Kriegsdienstverweigerer,Soldat,Zivildienstleistende,Einberufung


In [43]:
file = join(ETL_PATH, 'LDAmodel', 'noun', 'Reranker', 'dewac_nbfiles01_label-candidates_full.csv')
full = pd.read_csv(file)
full = full.rename(columns={'Unnamed: 6': 'label_method'})
#print(full.head())
simple = (
    full
    .query('metric == "ref" and param_id == "e42" and nb_topics == 100 and label_method == "comb"')
    .drop(full.columns[:7], axis=1)
    .reset_index(drop=True)
    .applymap(lambda x: eval(x)[0])
    .iloc[:20, :]
)
simple

,label0,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
0,kommunistisch_Partei,kommunistisch_Partei_Deutschland,Arbeiterpartei,sozialistisch_Partei,bürgerlich_Partei,Sozialdemokratie,Massenorganisation,frei_Gewerkschaft,demokratisch_Sozialismus,Socjaldemokracja_Rzeczypospolitej_Polskiej,sozialdemokratisch_Partei_Deutschland,sozialdemokratisch_Partei,Bolschewik,rot_Ruhrarmee,Kommunismus,Einheitsfront,Aktion_demokratisch_Fortschritt,sozialdemokratisch_Arbeiterpartei,gothaer_Programm,unabhängig_Gewerkschaftsopposition
1,Pittheu,Kretheu,Jüngling,Hreidmar,Argeia,Eriphyle,Iole,Euadne,Deiphontes,Deiphobo,Johann_von_England,nach_Haus,Knabe,Tochter_der_König,Ninigi,Chrysippos,Magd,Der_Vater,Stiefmutter,Anna
2,gleichgeschlechtlich_Ehe,Lebenspartnerschaftsgesetz,Verpartnerung,gleichgeschlechtlich,Lebensgemeinschaft,lesbisch,Scheidung,Eherecht,Gütertrennung,Homosexualität,Ehescheidung,Regenbogenfamilie,Lebensformenpolitik,Abstandsgebot,sexuell_Identität,Partnerschaftsgesetz,Eingetragene_Partnerschaft,Sexualpartner,Ehepakt,elterlich_Sorge
3,Personalvertretungsgesetz,Betriebsrat,Beschäftigte,Bundespersonalvertretungsgesetz,Gesamtbetriebsrat,Arbeitnehmer,Betriebsvereinbarung,Betriebsbuße,Drittelbeteiligungsgesetz,Weiterbeschäftigungsanspruch,Arbeitsrecht,Tarifvertragspartei,Tarifvertrag,Arbeitsverhältnis,Betriebsrentengesetz,Betriebsänderung,Belegschaft,Beschlussverfahren,Verbrauchervertrag,Kündigungsschutzgesetz
4,Aktionär,Vorstandsvorsitzende,Aktiengesellschaft,Anteilseigner,Grundkapital,Geschäftsanteil,Kapitalerhöhung,Gesellschafter,Stammaktie,One_Equity_Partner,Board_of_Director,Aufsichtsrat,Gesellschafterversammlung,Generalversammlung,Hauptversammlung,Altaktionär,Aktienrecht,operativ_Geschäft,Kommanditgesellschaft_auf_Aktie,Vorzugsaktie
5,Regierungschef,Legislative,Verfassung,Verfassungsrecht,Volksvertretung,Staatsrat,amerikanisch_Menschenrechtskonvention,Vorsitzende,Gesetzgebung,Judikative,Staatsoberhaupt,Rechtsgrundlage,Statut,Abkomme,Exekutive,Mitgliedstaat,Dekret,Verfassungsänderung,allgemein_Gewaltverbot,Senat
6,päpstlich_Legat,Weihbischof,Kurienbischof,Erzbistum_Turin,Suffraganbischof,Erzbischof,Erzpriester,Diözesanbischof,Bischof,apostolisch_Delegat,Metropolit,Dompropst,Pontifikat,Papst,Bischofssitz,Offizial,Diözese,Generalvikar,Kathedrale,James_Robert_Knox
7,Kläger,Prozessgericht,Hauptverfahren,Prozessurteil,Angeklagte,Zulässigkeit,Vollstreckbarkeit,Teilurteil,sofortig_Vollziehung,Ablehnungsbescheid,Gerichtsverfahren,Hauptsacheverfahren,sofortig_Beschwerde,Anhörung,Zwischenurteil,Haftbeschwerde,Endurteil,Unzulässigkeit,Sprungrevision,maßgeblich_Zeitpunkt
8,Angeklagte,Kläger,Hauptverfahren,Gerichtsverfahren,Verdächtige,Strafsache,Anschuldigung,Tatverdacht,Strafkammer,Beschuldigter,Indiz,Freispruch,Prozessgericht,Beweismittel,Generalstaatsanwaltschaft,Staatsschutzkammer,Ermittlungsverfahren,Zwischenurteil,Gerichtsverhandlung,Staatsanwalt
9,weiß_Jahrgang,Ersatzreserve,Wehrpflicht,Berufssoldat,Einberufung,Soldat_auf_Zeit,freiwillig_Wehrdienst,Soldatengesetz,Zivildienst,Wehrsold,Wehrersatzdienst,Zivildienstgesetz,Rekrut,Soldatenversorgungsgesetz,Reserveoffizier,Gebirgsjäger,Dreijährig-Freiwilliger,Totalverweigerung,Wehrpflicht_in_Deutschland,Wehrsoldgesetz


In [45]:
from train_utils import EpochLogger, EpochSaver

d2vpath = join(ETL_PATH, 'embeddings', 'd2v', 'd2v')
w2vpath = join(ETL_PATH, 'embeddings', 'w2v', 'w2v')
print("Doc2Vec loading", d2vpath)
d2v = Doc2Vec.load(d2vpath)

print("Word2Vec loading", w2vpath)
w2v = Word2Vec.load(w2vpath)
print("models loaded")

Doc2Vec loading ../data/preprocessed/embeddings/d2v/d2v
Word2Vec loading ../data/preprocessed/embeddings/w2v/w2v
models loaded


In [46]:
d2v_wv = d2v.wv
d2v_dv = d2v.docvecs
w2v_wv = w2v.wv

d2vmodel.infer_vector(simple) #infer a d2v representation for a topic

del d2v
del w2v

In [55]:
w1 = simple.iloc[0,0]
vec1 = d2v_wv[w1]
w2 = simple.iloc[0,1]
vec2 = d2v_wv[w2]

In [60]:
# w2v_wv.cosine_similarities(vec, d2v_wv.vectors)
d2v_dv.distance(w1, w2), d2v_wv.distance(w1, w2), w2v_wv.distance(w1, w2)

(0.4414312243461609, 0.4072136878967285, 0.3993687629699707)

In [70]:
dists = d2v_wv.distances(w1, tuple(simple.iloc[0, 1:].values))
simple.iloc[0, 1:].to_frame().assign(d2v_wv_distances=dists).sort_values('d2v_wv_distances')

,0,d2v_wv_distances
label3,sozialistisch_Partei,0.246073
label14,Kommunismus,0.261926
label15,Einheitsfront,0.272340
label2,Arbeiterpartei,0.280178
label11,sozialdemokratisch_Partei,0.335489
label6,Massenorganisation,0.358646
label8,demokratisch_Sozialismus,0.396195
label5,Sozialdemokratie,0.402398
label12,Bolschewik,0.403119
label1,kommunistisch_Partei_Deutschland,0.407214


In [72]:
dists = w2v_wv.distances(w1, tuple(simple.iloc[0, 1:].values))
simple.iloc[0, 1:].to_frame().assign(w2v_wv_distances=dists).sort_values('w2v_wv_distances')

,0,w2v_wv_distances
label3,sozialistisch_Partei,0.294670
label2,Arbeiterpartei,0.295048
label11,sozialdemokratisch_Partei,0.347479
label14,Kommunismus,0.375449
label1,kommunistisch_Partei_Deutschland,0.399369
label5,Sozialdemokratie,0.399683
label15,Einheitsfront,0.401898
label6,Massenorganisation,0.462755
label8,demokratisch_Sozialismus,0.465640
label7,frei_Gewerkschaft,0.470516


In [81]:
has_vec = simple.iloc[0, :].map(lambda x: x in d2v_dv)
has_vec

label0     True
label1     True
label2     True
label3     True
label4     True
label5     True
label6     True
label7     True
label8     True
label9     True
label10    True
label11    True
label12    True
label13    True
label14    True
label15    True
label16    True
label17    True
label18    True
label19    True
Name: 0, dtype: bool

In [85]:
for word in simple.iloc[0, :]:
    print(word, d2v_dv[word][:3])

kommunistisch_Partei [0.14880304 0.2410298  0.00921005]
kommunistisch_Partei_Deutschland [ 0.01485822  0.23265952 -0.18486771]
Arbeiterpartei [ 0.39894158 -0.55674595  0.18212381]
sozialistisch_Partei [ 0.19551352 -0.08076364 -0.11990631]
bürgerlich_Partei [ 0.2925751  -0.00560244  0.06508418]
Sozialdemokratie [-0.00299884 -0.6104501   0.39840788]
Massenorganisation [-0.08753942  0.16435364  0.19077021]
frei_Gewerkschaft [ 0.11946975 -0.03126214 -0.21551421]
demokratisch_Sozialismus [ 0.35326654  0.45461226 -0.3754114 ]
Socjaldemokracja_Rzeczypospolitej_Polskiej [-0.05088069 -0.22011547 -0.05530345]
sozialdemokratisch_Partei_Deutschland [0.17208889 0.14877    0.11397935]
sozialdemokratisch_Partei [-0.08320348  0.22613658 -0.13806887]
Bolschewik [ 0.02478602 -0.10575379 -0.42004907]
rot_Ruhrarmee [ 0.36459985  0.15176307 -0.10058993]
Kommunismus [-0.11453343  0.4268518   0.15717927]
Einheitsfront [ 0.5541612  -0.16723068 -0.54578424]
Aktion_demokratisch_Fortschritt [-0.05979657 -0.38087

In [110]:
dists = d2v_dv.distances(w1, tuple(simple.iloc[0, 1:].values))
simple.iloc[0, 1:].to_frame().assign(d2v_dv_distances=dists).sort_values('d2v_dv_distances')

TypeError: not all arguments converted during string formatting

In [116]:
# ! nice !
labels = simple.iloc[0].copy()
for i in range(10):
    outlier = w2v_wv.doesnt_match(labels)
    print(outlier)
    labels = labels[labels != outlier]
print(labels)

rot_Ruhrarmee
gothaer_Programm
Aktion_demokratisch_Fortschritt
unabhängig_Gewerkschaftsopposition
Massenorganisation
Bolschewik
Socjaldemokracja_Rzeczypospolitej_Polskiej
Kommunismus
demokratisch_Sozialismus
Einheitsfront
label0                      kommunistisch_Partei
label1          kommunistisch_Partei_Deutschland
label2                            Arbeiterpartei
label3                      sozialistisch_Partei
label4                         bürgerlich_Partei
label5                          Sozialdemokratie
label7                         frei_Gewerkschaft
label10    sozialdemokratisch_Partei_Deutschland
label11                sozialdemokratisch_Partei
label17        sozialdemokratisch_Arbeiterpartei
Name: 0, dtype: object


In [118]:
topic = topics.iloc[0]
w2v_wv.most_similar(positive=topic, negative=None, topn=20, restrict_vocab=None, indexer=None)
#wv.most_similar([rand_vec], topn=5, indexer=annoy_index)

[('kommunistisch', 0.7275704145431519),
 ('kommunistisch_Partei', 0.7142001390457153),
 ('Sozialdemokrat', 0.7077106237411499),
 ('Arbeiterpartei', 0.707636833190918),
 ('KPD', 0.702242374420166),
 ('Sozialist', 0.701751708984375),
 ('sozialistisch', 0.7008939981460571),
 ('Sozialdemokratie', 0.695747971534729),
 ('Linkspartei', 0.6750216484069824),
 ('sozialdemokratisch_Partei', 0.6743425130844116),
 ('sozialdemokratisch', 0.6684965491294861),
 ('Arbeiterbewegung', 0.6586674451828003),
 ('sozialistisch_Partei', 0.6555250883102417),
 ('Parteimitglied', 0.6434710621833801),
 ('Gewerkschafter', 0.6433238983154297),
 ('Parteiführung', 0.6424713134765625),
 ('Volksfront', 0.6394513845443726),
 ('Die_Partei', 0.6379992961883545),
 ('Gewerkschaftsbewegung', 0.6360800266265869),
 ('frei_Gewerkschaft', 0.632095456123352)]

In [123]:
w2v_wv.most_similar_cosmul(positive=list(topic), negative=None, topn=20)

[('kommunistisch', 0.03645792976021767),
 ('kommunistisch_Partei', 0.0343996100127697),
 ('Arbeiterpartei', 0.03342046961188316),
 ('Sozialdemokrat', 0.033226363360881805),
 ('sozialistisch', 0.032760392874479294),
 ('KPD', 0.03267296403646469),
 ('Sozialist', 0.03214901313185692),
 ('Sozialdemokratie', 0.03196638822555542),
 ('sozialdemokratisch_Partei', 0.0290193110704422),
 ('Linkspartei', 0.028594257310032845),
 ('sozialdemokratisch', 0.028360987082123756),
 ('Arbeiterbewegung', 0.027008317410945892),
 ('sozialistisch_Partei', 0.02643883228302002),
 ('Parteiführung', 0.02553098089993),
 ('Volksfront', 0.025441884994506836),
 ('Gewerkschafter', 0.025384627282619476),
 ('Parteimitglied', 0.02523638866841793),
 ('Gewerkschaftsbewegung', 0.024693815037608147),
 ('frei_Gewerkschaft', 0.0242183618247509),
 ('Die_Partei', 0.024198325350880623)]

In [154]:
w2v_scores = topic.to_frame().apply(lambda x: pd.Series(w2v_wv.similar_by_word(x[0], topn=20, restrict_vocab=None)), axis=1).stack().apply(pd.Series).rename(columns={0:'term', 1:'score'})
w2v_scores.groupby('term').agg(['count', 'sum']).sort_values(('score', 'sum'), ascending=False)

score          
                              count       sum
term                                         
kommunistisch                     3  1.916268
Sozialdemokrat                    3  1.888201
KPD                               3  1.810673
Kommunismus                       2  1.433982
Linkspartei                       2  1.380561
sozialistisch                     2  1.345965
Arbeiterpartei                    2  1.327648
sozialistisch_Partei              2  1.305735
Arbeiterbewegung                  2  1.258019
Gewerkschafter                    2  1.228253
kommunistisch_Partei              2  1.228026
Parteimitglied                    2  1.213721
Kpd-Mitglied                      2  1.157464
Trotzkist                         2  1.142914
SED                               2  1.079617
deutsch_demokratisch_Republik     1  0.862029
Die_Partei                        1  0.820931
kämpfen                           1  0.774020
gewerkschaftlich                  1  0.768169
Wahlbündnis                       1  0.767385
Bundesrepublik                    1  0.766973
Die_Linke                         1  0.766315
BRD                               1  0.760405
politisch_Partei                  1  0.755516
Oppositionspartei                 1  0.755359
Ost-Berlin                        1  0.754305
Marxismus                         1  0.753668
Volkspartei                       1  0.739776
Der_Kampf                         1  0.737193
Sozialist                         1  0.733743
...                             ...       ...
Die_Kraft                         1  0.513868
Thälmann                          1  0.504023
Parteigenosse                     1  0.500983
rot_Hilfe                         1  0.491420
Komintern                         1  0.485952
Kamerad                           1  0.483451
Gesetz                            1  0.479620
Liebknecht                        1  0.476753
Parteifunktionär                  1  0.474569
wirksam                           1  0.474116
treibend                          1  0.466206
Revolutionär                      1  0.466027
Bewegung                          1  0.461314
Ernst_Thälmann                    1  0.457790
Parteiführung                     1  0.457220
Wirkung                           1  0.456929
Zk-Mitglied                       1  0.456518
anderer_Kraft                     1  0.432759
Gegenkraft                        1  0.418986
Macht                             1  0.418620
tretend                           1  0.418179
Inkrafttreten                     1  0.414207
Stärke                            1  0.412680
kraft                             1  0.406520
Widerstand                        1  0.406055
Das_Gesetz                        1  0.403749
Zwangskraft                       1  0.401773
Wirkt                             1  0.401426
Energie                           1  0.400949
Widerstandskraft                  1  0.400275

[182 rows x 2 columns]

In [156]:
d2v_scores = topic.to_frame().apply(lambda x: pd.Series(d2v_wv.similar_by_word(x[0], topn=20, restrict_vocab=None)), axis=1).stack().apply(pd.Series).rename(columns={0:'term', 1:'score'})
d2v_scores.groupby('term').agg(['count', 'sum']).sort_values(('score', 'sum'), ascending=False)

score          
                              count       sum
term                                         
Kommunismus                       2  1.574009
sozialistisch                     2  1.552963
kommunistisch                     2  1.500283
gewerkschaftlich                  2  1.488509
Sozialist                         2  1.485968
Arbeiterbewegung                  2  1.427791
Streik                            2  1.414265
demokratisch_Sozialismus          2  1.402648
Arbeitskampf                      2  1.362184
Trotzkist                         2  1.340160
Kpd-Mitglied                      2  1.305032
KPD                               2  1.304928
Kommunistin                       2  1.292288
kämpfen                           1  0.885046
deutsch_demokratisch_Republik     1  0.856135
Die_Linke                         1  0.853391
gewerkschaftliche                 1  0.845275
Marxismus                         1  0.839779
Die_Partei                        1  0.836632
kommunistisch_Partei              1  0.831114
Parteivorsitzender                1  0.828297
marxistisch                       1  0.821943
Gewerkschaftsmitglied             1  0.818353
kandidieren                       1  0.818348
Mandat                            1  0.817909
Parlament                         1  0.814652
Kapitalismus                      1  0.811166
Wahlbündnis                       1  0.810763
WASG                              1  0.806454
parteiintern                      1  0.803055
...                             ...       ...
lösen                             1  0.601118
denunzieren                       1  0.596055
entgegen                          1  0.595418
fordern                           1  0.594669
erklären                          1  0.589352
Widerstand                        1  0.587902
Bewegung                          1  0.580737
verhaften                         1  0.576548
Verhaftung                        1  0.573729
konterrevolutionär                1  0.573209
Parteiarbeit                      1  0.572995
indem                             1  0.572355
Verhaftungswelle                  1  0.571892
Zentralkomitee                    1  0.571363
Kpd-Funktionär                    1  0.570785
Zuchthausstrafe                   1  0.570764
Sympathisant                      1  0.569994
dadurch                           1  0.568401
Hochverrat                        1  0.567586
verhindern                        1  0.565758
gemäß                             1  0.563363
Wille                             1  0.559268
versuchen                         1  0.558938
geschehen                         1  0.556896
Aufgabe                           1  0.550651
ausüben                           1  0.549721
unabhängig                        1  0.547636
ändern                            1  0.545129
Weise                             1  0.542842
getreten                          1  0.542773

[187 rows x 2 columns]

In [162]:
# combined
comb_scores = w2v_scores.append(d2v_scores).groupby('term').agg(['count', 'sum']).sort_values(('score', 'sum'), ascending=False)
comb_scores

score          
                              count       sum
term                                         
kommunistisch                     5  3.416551
KPD                               5  3.115601
Kommunismus                       4  3.007992
sozialistisch                     4  2.898928
Arbeiterbewegung                  4  2.685810
Sozialdemokrat                    4  2.620496
Trotzkist                         4  2.483075
Kpd-Mitglied                      4  2.462496
gewerkschaftlich                  3  2.256677
Sozialist                         3  2.219711
Linkspartei                       3  2.145905
Streik                            3  2.064514
kommunistisch_Partei              3  2.059141
demokratisch_Sozialismus          3  2.018154
Gewerkschafter                    3  2.007068
Arbeitskampf                      3  1.968626
deutsch_demokratisch_Republik     2  1.718164
kämpfen                           2  1.659066
Die_Partei                        2  1.657563
Die_Linke                         2  1.619706
Marxismus                         2  1.593447
Wahlbündnis                       2  1.578149
gewerkschaftliche                 2  1.563624
politisch_Partei                  2  1.552476
Bundesrepublik                    2  1.542400
Parteivorsitzender                2  1.539072
Kapitalismus                      2  1.537759
WASG                              2  1.532479
Ost-Berlin                        2  1.531346
BRD                               2  1.525272
...                             ...       ...
Gegenangriff                      1  0.554092
Aufgabe                           1  0.550651
ausüben                           1  0.549721
unabhängig                        1  0.547636
ändern                            1  0.545129
Weise                             1  0.542842
Die_Kraft                         1  0.513868
Thälmann                          1  0.504023
Parteigenosse                     1  0.500983
Kamerad                           1  0.483451
Liebknecht                        1  0.476753
Parteifunktionär                  1  0.474569
wirksam                           1  0.474116
treibend                          1  0.466206
Revolutionär                      1  0.466027
Ernst_Thälmann                    1  0.457790
Wirkung                           1  0.456929
Zk-Mitglied                       1  0.456518
anderer_Kraft                     1  0.432759
Gegenkraft                        1  0.418986
Macht                             1  0.418620
tretend                           1  0.418179
Inkrafttreten                     1  0.414207
Stärke                            1  0.412680
kraft                             1  0.406520
Das_Gesetz                        1  0.403749
Zwangskraft                       1  0.401773
Wirkt                             1  0.401426
Energie                           1  0.400949
Widerstandskraft                  1  0.400275

[276 rows x 2 columns]

In [164]:
comb_scores[comb_scores.index.isin(phrases.token)]

score          
                                    count       sum
term                                               
Kommunismus                             4  3.007992
Arbeiterbewegung                        4  2.685810
Linkspartei                             3  2.145905
Streik                                  3  2.064514
kommunistisch_Partei                    3  2.059141
demokratisch_Sozialismus                3  2.018154
Arbeitskampf                            3  1.968626
deutsch_demokratisch_Republik           2  1.718164
Die_Partei                              2  1.657563
Die_Linke                               2  1.619706
Marxismus                               2  1.593447
Wahlbündnis                             2  1.578149
politisch_Partei                        2  1.552476
Parteivorsitzender                      2  1.539072
Kapitalismus                            2  1.537759
Ost-Berlin                              2  1.531346
BRD                                     2  1.525272
Der_Kampf                               2  1.504382
Bündnis_90/Die_Grüne                    2  1.492059
Partei_der_demokratisch_Sozialismus     2  1.475292
deutsch_Wiedervereinigung               2  1.469624
Klassenkampf                            2  1.420565
frei_Gewerkschaft                       2  1.417674
Ideologie                               2  1.396868
Sozialdemokratie                        2  1.395365
FDP                                     2  1.393800
Staatssozialismus                       2  1.390718
Westdeutschland                         2  1.385249
Diktatur_der_Proletariat                2  1.375059
Gewerkschaftsbund                       2  1.370322
...                                   ...       ...
IG_Metall                               1  0.590126
Arbeiterin                              1  0.586439
frei_Wähler                             1  0.585403
Arbeitnehmer                            1  0.580142
Belegschaft                             1  0.578942
Tagelöhner                              1  0.577690
Verhaftung                              1  0.573729
Zentralkomitee                          1  0.571363
Feldzug                                 1  0.570981
Sympathisant                            1  0.569994
Hochverrat                              1  0.567586
Facharbeiter                            1  0.564527
Auseinandersetzung                      1  0.562895
Scharmützel                             1  0.561986
Wille                                   1  0.559268
Straßenkampf                            1  0.558997
Aufgabe                                 1  0.550651
unabhängig                              1  0.547636
Weise                                   1  0.542842
Die_Kraft                               1  0.513868
Kamerad                                 1  0.483451
Liebknecht                              1  0.476753
Ernst_Thälmann                          1  0.457790
Wirkung                                 1  0.456929
anderer_Kraft                           1  0.432759
Macht                                   1  0.418620
Stärke                                  1  0.412680
Das_Gesetz                              1  0.403749
Zwangskraft                             1  0.401773
Energie                                 1  0.400949

[130 rows x 2 columns]

In [ ]:
w2v_wv.similarity(w1, w2)